Aging Provision Policy – Additional Provision Calculation 

Policy Overview
To ensure prudent financial coverage, additional provisions are calculated for inventory in special categories, using the following rules:

1. Damaged Inventory
Provision Rate:
100% of the net total cost of damaged stock.
2. Leftover Inventory
For Closed Brands:
50% of the net total cost of leftover stock from brands that are closed.
For Running Brands:
15% of the net total cost of leftover stock from brands that are still operating.
3. Closed Brands (Other Locations)
Provision Rate:
50% of the net total cost of stock from closed brands (excluding leftover and damaged locations).
4. Exclusions
No additional provision is made for inventory under consignment, guaranteed margin, or “Buying Pull - Mango” models.
Summary:
These additional provisions are applied on top of the standard aging policy to ensure adequate financial coverage for higher-risk inventory categories.

1. update the closed brands status in the mapping file
2. check the totals of the net cost throughout the steps to ensure the completness
3. update the combinations in the combination file
4. check missing standard brands in the mapping file and soh
5. check std season and buckets in the checks section of the code
6. update the parameters below
7. check for garbage std season in the code
8. ensure no duplicates in the combination[std brand name and location name] and mapping file [original brand name]
 

In [3]:
import pandas as pd
import numpy as np

AttributeError: partially initialized module 'pandas' has no attribute '_pandas_parser_CAPI' (most likely due to a circular import)

parameters

In [ ]:

SOH_File = 'SOH-mar.xlsx'
first_first_bucket_number_seasons = 5
damage_percentage = 1
leftover_running_percentage = .15
leftover_closed_percentage = .5
closed_percentage = .5

brand_specific_provision = {}
'''    'FLYING TIGER': 2
}'''

"    'FLYING TIGER': 2\n}"

In [ ]:
import re

def season_sort_key(season):
    if not isinstance(season, str) or len(season) < 4:
        return (0, 0)  # Push unknowns to bottom

    season_type = season[:2]  # 'AW' or 'SS'
    year = int(season[-2:])

    # AW should come *before* SS, so give it a higher priority
    season_rank = 1 if season_type == "AW" else 0

    return (year, season_rank)

def standardize_season(raw_season):
    if not isinstance(raw_season, str) or raw_season.strip() == "":
        return "Unknown"

    season = raw_season.strip().upper()

    # --- Priority: continuity/old ---
    if "CONTINUITY" in season or "BASICS" in season:
        return "Continuity"

    elif "OLD" in season:
        return "Old-"

    # --- 4-digit year + seasonal tag ---
    year_match = re.search(r"(20\d{2})", season)
    if year_match:
        year = year_match.group(1)[2:]  # Take last 2 digits, e.g., '2023' → '23'

        if any(tag in season for tag in ["SPRING", "SUMMER", "SS"]):
            return f"SS{year}"
        elif any(tag in season for tag in ["AUTUMN", "WINTER", "AW"]):
            return f"AW{year}"

    # --- 2-digit season pattern like AW23, SS22 ---
    match = re.search(r"(SS|AW)(\d{2})", season)
    if match:
        return f"{match.group(1)}{match.group(2)}"

    # --- WA fallback to AW ---
    if "WA" in season:
        match = re.search(r"WA(\d{2})", season)
        if match:
            return f"AW{match.group(1)}"

    # --- Last resort: any 2-digit year with best guess ---
    match = re.search(r"(\d{2})", season)
    if match:
        return f"SS{match.group(1)}"
    
    return "Unknown"  # default if nothing matches


aging provision as per policy

In [ ]:
soh = pd.read_excel(SOH_File, sheet_name='Sheet1')
soh = soh[(soh['GROUP_NAME'] != 'Aleph' )& (soh['AR Comments'] == 'Consider')]
existing_balances = pd.read_excel('current_balance.xlsx', sheet_name='Sheet1')
mapping = pd.read_excel('mapping.xlsx', sheet_name='Sheet1')
soh['NETTOTAL_COST'].fillna(0, inplace=True)
soh['NETTOTAL_COST'] = pd.to_numeric(soh['NETTOTAL_COST'], errors='coerce')

original_season = 'SEASON_DESC' if 'SEASON_DESC' in soh.columns else 'SEASON DESC'
soh['NETTOTAL_COST'].sum()


C:\Users\saad.qureshi\AppData\Local\Temp\ipykernel_31956\3354986209.py:5: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  soh['NETTOTAL_COST'].fillna(0, inplace=True)


418141106.11822

In [ ]:
combinations = pd.read_excel('combinations.xlsx', sheet_name='Sheet1')

In [ ]:
soh['GROUP_NAME'] = soh['GROUP_NAME'].str.upper()
mapping['GROUP_NAME'] = mapping['GROUP_NAME'].str.upper()
soh = soh.merge(mapping, on='GROUP_NAME', how='left')
#soh['Std Brand'] = np.where(soh['Std Brand'].isna(), soh['GROUP_NAME'], soh['Std Brand'])
soh = soh[(soh['Closed_status'] != 'Exit' )]
soh['NETTOTAL_COST'].sum()

418141106.11822

In [ ]:
'''valid_brands = soh.groupby('Std Brand')['NETTOTAL_COST'].sum()
valid_brands = valid_brands[valid_brands != 0].index

# Step 2: filter original DataFrame
soh = soh[soh['Std Brand'].isin(valid_brands)]
soh['NETTOTAL_COST'].sum()'''

"valid_brands = soh.groupby('Std Brand')['NETTOTAL_COST'].sum()\nvalid_brands = valid_brands[valid_brands != 0].index\n\n# Step 2: filter original DataFrame\nsoh = soh[soh['Std Brand'].isin(valid_brands)]\nsoh['NETTOTAL_COST'].sum()"

In [ ]:
soh['std_season'] = soh[original_season].apply(standardize_season)

In [ ]:
excluded = {'Unknown', 'Continuity', 'Old-','AW97'}
unique_season = [f for f in soh['std_season'].dropna().unique() if f not in excluded]

In [ ]:
sorted_std_season_in_soh = sorted(unique_season, key=season_sort_key, reverse=True)

In [ ]:
bucket1 = sorted_std_season_in_soh[:first_first_bucket_number_seasons]
bucket2 = sorted_std_season_in_soh[first_first_bucket_number_seasons:first_first_bucket_number_seasons + 3]
bucket3 = sorted_std_season_in_soh[first_first_bucket_number_seasons+3:first_first_bucket_number_seasons + 6] 
bucket4 = sorted_std_season_in_soh[first_first_bucket_number_seasons + 6:]

In [ ]:
bucket1

['AW25', 'SS25', 'AW24', 'SS24', 'AW23']

In [ ]:
bucket1 =   bucket1 + ['Unknown','Continuity'] 
bucket4 = bucket4 + [  'Old-','AW97'] 

In [ ]:

conditions = [
    soh['std_season'].isin(bucket1),
    soh['std_season'].isin(bucket2),
    soh['std_season'].isin(bucket3)
]

choices = ['bucket1', 'bucket2', 'bucket3']

soh['season_bucket'] = np.select(conditions, choices, default='bucket4')

In [ ]:
soh['Continuity_factor'] = 0.40
soh['provision_%_policy'] = soh['season_bucket'].map({
    'bucket1': 0,
    'bucket2': 0.15,
    'bucket3': 0.50,
    'bucket4': 0.75  # Assuming bucket4 is the default for other buckets

})  # Or any default value for other buckets like 'bucket3', 'bucket4'

In [ ]:

soh.loc[
    soh['Model'].isin([
        'Consignment',
        'Guaranteed Margin',
        'Buying Pull - Mango'
    ]),
    ['provision_amount_policy', 'provision_%_policy', 'Continuity_factor']
] = 0

soh['provision_amount_policy'] = soh['NETTOTAL_COST'] * soh['provision_%_policy'] * soh['Continuity_factor']

Damage locations


In [ ]:
soh['location_catergory'] = "Store, Online & WH"

soh.loc[
    soh['LOCATION_NAME'].astype(str).str.lower().str.contains('leftover', na=False),
    "location_catergory"
] = "Leftover"
soh.loc[soh['LOCATION_NAME'].astype(str).str.lower().str.contains('damage', na=False),"location_catergory"] = "Damage"
soh.loc[soh['LOCATION_NAME'].astype(str).str.lower().str.contains('sulay', na=False),"location_catergory"] = "Leftover"

In [ ]:
condition = soh['location_catergory'] == "Damage"  


soh.loc[condition, 'additional_provision'] = (soh["NETTOTAL_COST"] * damage_percentage) - soh['provision_amount_policy']



Leftover

In [ ]:

#Leftover of the closed brands
condition =(
    (soh['Closed_status'] == "Closed") & 
    ( soh['location_catergory'] == "Leftover" ) 
    ) 

soh.loc[condition, 'additional_provision'] = (soh["NETTOTAL_COST"] * leftover_closed_percentage) - soh['provision_amount_policy']

condition =(
    (soh['Closed_status'] != "Closed") & 
    ( soh['location_catergory'] == "Leftover" ) 
    ) 

soh.loc[condition, 'additional_provision'] = (soh["NETTOTAL_COST"] * leftover_running_percentage) - soh['provision_amount_policy']


Closed brands

In [ ]:
condition = (
    (soh['Closed_status'] == "Closed") & 
    (~soh['location_catergory'].isin(["Leftover","Damage"] )) 
    )
soh.loc[condition, 'additional_provision'] =    (soh["NETTOTAL_COST"] * closed_percentage) - soh['provision_amount_policy']

Brand specific provisions

In [ ]:
if brand_specific_provision:
    soh['additional_provision'] = np.where(
        soh['Std Brand'].isin(brand_specific_provision.keys()),
        soh['NETTOTAL_COST'] * soh['Std Brand'].map(brand_specific_provision) - soh['provision_amount_policy'],
        soh['additional_provision']
    )

removing the consginement and gurantee margin provision

In [ ]:
soh.loc[
    soh['Model'].isin([
        'Consignment',
        'Guaranteed Margin',
        'Buying Pull - Mango'
    ]),
    ['provision_amount_policy', 'provision_%_policy', 'Continuity_factor','additional_provision']
] = 0

soh['provision_amount_policy'] = soh['provision_amount_policy'].fillna(0)
soh['additional_provision'] = soh['additional_provision'].fillna(0)


soh['Total Provision'] = soh['provision_amount_policy'] + soh['additional_provision']

In [ ]:
soh.to_csv(r"Output/aging_provision.csv")

Summary

In [ ]:
summary = soh.groupby(by='Std Brand')[["NETTOTAL_COST",'provision_amount_policy','additional_provision','Total Provision']].sum()
summary['coverage'] = summary['Total Provision'] / summary['NETTOTAL_COST']

Combinations

In [ ]:
soh_with_combinations = soh.merge(combinations, on=['Std Brand','LOCATION'], how='left')
soh_with_combinations[['s1', 's2', 's3', 's4']]= soh_with_combinations[['s1', 's2', 's3', 's4']].fillna(0)
soh_with_combinations.to_excel(r"Output/aging_provision_combinations.xlsx", index=False)

checks

In [ ]:
# verify the original seasons in the std buckets 
check_buckets = soh[['season_bucket','std_season' ]].drop_duplicates()

In [ ]:
# verify the seasons in the correct buckets 
check_season = soh[['std_season', original_season]].drop_duplicates()

In [ ]:
#missing combinations in output
missing_combinations = soh_with_combinations[(soh_with_combinations['NETTOTAL_COST'] != 0)&(soh_with_combinations['s1'].isna())] 

In [ ]:
# 1. Check for missing values in key columns
missing_in_soh = soh[soh[['Std Brand', 'LOCATION_NAME', 'NETTOTAL_COST', 'std_season']].isnull().any(axis=1)]
print("Missing values in SOH key columns:", missing_in_soh.shape[0])


# 3. Check for duplicates in mapping [original brand name]
duplicates_mapping = mapping.duplicated(subset=['GROUP_NAME'], keep=False)
print("Duplicate original brand names in mapping:", mapping[duplicates_mapping].shape[0])

# 4. Check for missing standard brands in mapping and SOH
missing_std_brands_in_mapping = set(soh['Std Brand']) - set(mapping['Std Brand'])
print("Std Brands in SOH missing from mapping:", missing_std_brands_in_mapping)

missing_std_brands_in_soh = set(mapping['Std Brand']) - set(soh['Std Brand'])
print("Std Brands in mapping missing from SOH:", missing_std_brands_in_soh)

# 5. Check for garbage/unknown seasons in std_season
garbage_seasons = soh[soh['std_season'].isin(['Unknown', 'Old-', 'AW97'])]
print("Rows with garbage/unknown std_season:", garbage_seasons.shape[0])

# 6. Check for missing in combinations merge
missing_comb_rows = soh_with_combinations[soh_with_combinations['s1'].isnull()]
print("Rows missing combination mapping:", missing_comb_rows.shape[0])


Missing values in SOH key columns: 18
Duplicate original brand names in mapping: 0
Std Brands in SOH missing from mapping: {nan}
Std Brands in mapping missing from SOH: {'LA VIE EN ROSE', 'MONSOON ACCESSORIZE', 'CHARLES & KEITH', 'MACHKA', 'ESTEE LAUDER', "WOMEN'S SECRET", 'ADL', 'ZIDDY KIDS', 'IPEKYOL', 'LASENZA', 'PEDRO', 'NINE WEST', 'FLORMAR', 'ALDO & ALDO ACCS', 'SPRING', 'SPRINGFIELDS', 'OXXO', 'MARIE FRANCE', 'TWIST'}
Rows with garbage/unknown std_season: 1195
Rows missing combination mapping: 0


In [ ]:
soh_with_combinations[soh_with_combinations[original_season].isna() | (soh_with_combinations[original_season] == '')].groupby('Std Brand')['NETTOTAL_COST'].sum()

Std Brand
BERSHKA         -5.829510e+05
Decathlon       -1.863420e+06
LEFTIES         -1.507027e+06
MASSIMO DUTTI    1.728737e+06
OYSHO            2.584268e+05
PULL AND BEAR    1.587361e+06
STRADIVARIOUS   -2.256067e+05
ZARA            -1.480180e+07
ZARA HOME        1.027013e+05
Name: NETTOTAL_COST, dtype: float64

Preparation of the entry

In [ ]:
soh_with_combinations['Total Provision'].sum()

31003352.632795796

In [ ]:
entry = soh_with_combinations.groupby(["s1","s2","s3","s4"])['Total Provision'].sum().reset_index().fillna(0)
entry['s5'] = 63002


In [ ]:
entry['Total Provision'].sum()

31003352.6327958

In [ ]:
entry2 = entry.copy()
entry2['Total Provision'] = entry2['Total Provision'] * -1
entry2['s5'] = 23993

In [ ]:
completed_entry = pd.concat([entry, entry2], ignore_index=True)
completed_entry.rename(columns={'Total Provision': 'Dr/(CR)'}, inplace=True)
completed_entry = completed_entry[completed_entry['Dr/(CR)'] != 0]

In [ ]:
completed_entry[['s1', 's2', 's3', 's4', 's5','Dr/(CR)']].to_csv(r"Output/completed_entry.csv", index=False)

difference entry

In [ ]:
diff_table = soh_with_combinations.groupby(["s1","s2","s3","s4"])['Total Provision'].sum().reset_index().fillna(0).merge(existing_balances, on=['s1','s2','s3','s4'], how='outer').fillna(0)

In [ ]:
diff_table['Dr/(CR)'] = (diff_table['Total Provision'] + diff_table['Closing balance'])*-1
diff_table.drop(['Closing balance','Total Provision'], inplace=True,axis=1)

In [ ]:
diff_table['s5'] = 23993

In [ ]:
diff_table2 = entry.copy()
diff_table2['Dr/(CR)'] = diff_table2['Dr/(CR)'] * -1
diff_table2['s5'] = 63002

In [ ]:
diff_entry = pd.concat([diff_table, diff_table2], ignore_index=True)
#diff_entry.rename(columns={'Total Provision': 'Dr/(CR)'}, inplace=True)
diff_entry = diff_entry[diff_entry['Dr/(CR)'] != 0]

In [ ]:
diff_entry[['s1', 's2', 's3', 's4', 's5','Dr/(CR)']].to_csv(r"Output/diff_entry.csv", index=False)

Analysis

In [ ]:
damage_summary = soh_with_combinations[soh_with_combinations['location_catergory'] == 'Damage'].groupby('Std Brand')[['NETTOTAL_COST', 'Total Provision']].sum()
damage_summary['coverage'] = damage_summary['Total Provision'] / damage_summary['NETTOTAL_COST']
damage_summary

,NETTOTAL_COST,Total Provision,coverage
Std Brand,,,
ALO YOGA,2.373011e+04,2.373011e+04,1.0
BERSHKA,6.933447e+05,6.933447e+05,1.0
CLARKS,6.412972e+04,6.412972e+04,1.0
DESIGUAL,4.884041e+03,0.000000e+00,0.0
FG4 KIDS,1.796662e+05,1.796662e+05,1.0
FG4 WOMEN,2.609739e+05,2.609739e+05,1.0
FLYING TIGER,1.774563e+04,1.774563e+04,1.0
GAP,1.751902e+05,1.751902e+05,1.0
JACADI,3.281743e+04,0.000000e+00,0.0


In [ ]:
leftover_summary = soh_with_combinations[soh_with_combinations['location_catergory'] == 'Leftover'].groupby('Std Brand')[['NETTOTAL_COST', 'Total Provision']].sum()
leftover_summary['coverage'] = leftover_summary['Total Provision'] / leftover_summary['NETTOTAL_COST']
leftover_summary

,NETTOTAL_COST,Total Provision,coverage
Std Brand,,,
ALO YOGA,2.228423e+04,1.114211e+04,0.50
ANOTAH,2.452000e+01,1.226000e+01,0.50
BERSHKA,2.182564e+06,3.273846e+05,0.15
BOBBI BROWN,0.000000e+00,0.000000e+00,NaN
CLARKS,7.080837e+03,3.540418e+03,0.50
DESIGUAL,5.861895e+03,0.000000e+00,0.00
ELEMENT,2.366146e+04,1.183073e+04,0.50
FG4 KIDS,2.071485e+06,1.035742e+06,0.50
FG4 WOMEN,6.481480e+05,3.240740e+05,0.50


In [ ]:
closed_summary = soh_with_combinations[soh_with_combinations['Closed_status'] == 'Closed'].groupby('Std Brand')[['NETTOTAL_COST', 'Total Provision']].sum()
closed_summary['coverage'] = closed_summary['Total Provision'] / closed_summary['NETTOTAL_COST']
closed_summary

,NETTOTAL_COST,Total Provision,coverage
Std Brand,,,
AL MALKI,0.000000e+00,0.000000e+00,NaN
ALO YOGA,4.601434e+04,3.487223e+04,0.757856
ANOTAH,2.451530e+01,1.225765e+01,0.500000
BOBBI BROWN,0.000000e+00,0.000000e+00,NaN
CLARKS,7.121056e+04,6.767014e+04,0.950282
DESIGUAL,1.074594e+04,0.000000e+00,0.000000
ELEMENT,2.366146e+04,1.183073e+04,0.500000
FG4 KIDS,2.263008e+06,1.221337e+06,0.539696
FG4 WOMEN,9.090618e+05,5.850178e+05,0.643540
